In [2]:
!pip install pylance vectordb

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 16.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 22.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 8.7 MB/s et

In [3]:
import lancedb
uri = "data/sample-lancedb"
db = lancedb.connect(uri)


In [4]:
import pyarrow as pa

# Define schema for RefSeq
ref_seq_schema = pa.struct([
    ('name', pa.string()),
    ('name2', pa.string()),
    ('exonicAlleleFunction', pa.string())
])

# Define schema for Variant
variant_schema = pa.struct([
    ('chrom', pa.string()),
    ('pos', pa.int32()),
    ('alt', pa.string()),
    ('ref', pa.string()),
    ('refSeq', ref_seq_schema)
    ('refSeq', ref_seq_schema)
])

# Create sample data for RefSeq
sample_ref_seq = pa.array([
    {'name': 'seq1', 'name2': 'name2_1', 'exonicAlleleFunction': 'func1'},
    {'name': 'seq2', 'name2': 'name2_2', 'exonicAlleleFunction': 'func2'},
], type=ref_seq_schema)

# Create sample data for Variant
sample_variant = pa.array([
    {'chrom': 'chr1', 'pos': 12345, 'alt': 'A', 'ref': 'T', 'refSeq': {'name': 'seq1', 'name2': 'name2_1', 'exonicAlleleFunction': 'func1'}},
    {'chrom': 'chr2', 'pos': 67890, 'alt': 'G', 'ref': 'C', 'refSeq': {'name': 'seq2', 'name2': 'name2_2', 'exonicAlleleFunction': 'func2'}},
], type=variant_schema)

# Optionally, you can create an Arrow Table to store this data
arrow_table = pa.table({'variants': sample_variant})

print("Schema:", arrow_table.schema)
print("Data:", arrow_table.to_pandas())

Schema: variants: struct<chrom: string, pos: int32, alt: string, ref: string, refSeq: struct<name: string, name2: string, exonicAlleleFunction: string>>
  child 0, chrom: string
  child 1, pos: int32
  child 2, alt: string
  child 3, ref: string
  child 4, refSeq: struct<name: string, name2: string, exonicAlleleFunction: string>
      child 0, name: string
      child 1, name2: string
      child 2, exonicAlleleFunction: string
Data:                                             variants
0  {'chrom': 'chr1', 'pos': 12345, 'alt': 'A', 'r...
1  {'chrom': 'chr2', 'pos': 67890, 'alt': 'G', 'r...


In [5]:
table = db.create_table("iterable_table", data=arrow_table, schema=variant_schema)

In [6]:
table

LanceTable(iterable_table)

In [8]:
table2 = db.open_table("iterable_table")

In [9]:
table2.where("chrom == 'chr1'") 

AttributeError: 'LanceTable' object has no attribute 'where'

In [10]:
!pip install duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 6.7 MB/s eta 0:00:00m eta 0:00:010:00:01
